<a href="https://colab.research.google.com/github/LightYe4r/stock_data_analysis-SDA-/blob/master/%EC%9D%B4%EB%B3%B4%EB%A6%BC_%EC%A3%BC%EC%8B%9D%EC%B0%A8%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

아래 설치 후 런타임 재시작(세션 재시작)

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# 네이버 증권에서 반도체 관련 종목 가져오기

- Google Drive 마운트 하기
- Google Drive에서 Colab Notebooks 폴더 > 반도체주식 폴더 생성 후 네이버증권_반도체종목.txt 파일 위치하기

In [ ]:
stock = {}
with open('/content/drive/MyDrive/Colab Notebooks/반도체주식/네이버증권_반도체종목.txt','r') as f:
  lines = f.readlines()
  print(lines)
  for idx, line in enumerate(lines[::6]):
    stock[line.strip()] = lines[idx*6+1].strip()
print(stock)

In [ ]:
print(len(stock))

In [ ]:
#!pip install finance-datareader
#!pip install beautifulsoup4
#!pip install matplotlib
#!pip install pandas_ta
#!pip install plotly==5.11.0
#!pip install pandas_datareader

In [ ]:
import FinanceDataReader as fdr
import pandas as pd
import pandas_datareader.data as pdr
import pandas_ta as ta
import numpy as np
from matplotlib import gridspec
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')


from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# pre-processing

In [ ]:
stock_data = {}
def downlaod_stockdata():
  global stock_data
  for value in stock.values():
    df = fdr.DataReader(value)
    stock_data[value] = df

krx = fdr.StockListing('KRX')
downlaod_stockdata()

def get_stockdata(stock_code):
  global stock_data
  # 개장하지 않은 날 0인 데이터를 전 날 데이터로 채움
  stock_data[stock_code].replace(0,np.nan,inplace=True)
  stock_data[stock_code] = stock_data[stock_code].ffill()
  return stock_data[stock_code]
  # stock_data = pdr.get_data_yahoo(stock_code+'.KQ')
  # if stock_data.empty:
  #   stock_data = pdr.get_data_yahoo(stock_code+'.KS')

In [ ]:
buttons = []
for val in stock.keys():
  button = dict()
  button['args'] = ['type',val]
  button['label'] = val
  button['method'] = 'update'
  buttons.append(button)

# 재무제표와 합치기

In [ ]:
stock_df.loc[:datetime.strptime('2023/04','%Y/%m'),'매출액']

In [ ]:
len(list(df.index))

In [ ]:
file_path = '/content/drive/MyDrive/Colab Notebooks/반도체주식/data_csv/'
def merge_files(col_name):
  for name, value in stock.items():
    df = pd.read_csv(file_path+'A'+value+'.csv',header = 0,index_col=0)
    stock_df = stock_data[value]

    reverse_idx_date = ''
    for idx, idx_date in enumerate(list(df.index)):
      if idx == 0:
        stock_df.loc[:datetime.strptime(idx_date,'%Y/%m')+relativedelta(months=1),col_name] = df.loc[idx_date,col_name]
        reverse_idx_date = idx_date
      elif idx_date == list(df.index)[-1] :
        stock_df.loc[datetime.strptime(idx_date,'%Y/%m'):,col_name] = df.loc[idx_date,col_name]
      else:
        stock_df.loc[datetime.strptime(reverse_idx_date,'%Y/%m')+relativedelta(months=1):datetime.strptime(idx_date,'%Y/%m')+relativedelta(months=1),col_name] = df.loc[idx_date,col_name]
        reverse_idx_date = idx_date
    print(stock_df)


In [ ]:
ROA = 'ROA(%)(당기순이익(연율화) / 총자산(평균)) * 100ROA'
ROE = 'ROE(%)(지배주주순이익(연율화) / 지배주주지분(평균)) * 100ROE'
EPS = 'EPS(원)지배주주순이익 / 수정평균주식수EPS(원)'
BPS = 'BPS(원)지배주주순자산(자사주차감전) / 수정기말주식수BPS(원)'
PER = 'PER(배)수정주가(보통주) / 수정EPSPER'
PBR = 'PBR(배)수정주가(보통주) / 수정BPSPBR'
stock_num = '발행주식수'


financial_statement_list = ['매출액','영업이익','당기순이익'
                        ,'자산총계','부채총계','자본총계',ROA
                            ,ROE,EPS,BPS,PER,PBR,stock_num]
for f in financial_statement_list:
  merge_files(f)

In [ ]:
def drop_na(stock_code):
  get_stockdata(stock_code).dropna(axis=1,inplace=True)



In [ ]:
drop_na('389020')
stock_data['389020']

In [ ]:
def drop_columns(columns):
  for col in columns:
    for name, value in stock.items():
      stock_data[value]


# visualization

In [ ]:
def match_dfshape(df, start_date, end_date):
    # 주어진 기간 동안의 모든 날짜 생성
    expected_dates = pd.date_range(start=start_date, end=end_date)

    # 주식 DataFrame에 없는 날짜에 대한 데이터 생성
    missing_dates_data = pd.DataFrame(index=expected_dates, columns=df.columns)

    # 주식 DataFrame과 결합하여 결측치를 채움
    filled_df = pd.concat([df, missing_dates_data]).sort_index()

    filled_df.fillna(method='ffill', inplace=True)  # 이전 값으로 결측치 채우기
    filled_df.fillna(0, inplace=True)
    # 주어진 기간 동안의 데이터 추출
    filtered_data = filled_df.loc[start_date:end_date]

    return filtered_data

def cal_sma_ema_rsi(df):
  df['Change']=df['Close']-df['Open']
  df.loc[df['Change']>=0,'Volumecolor']=1
  df.loc[df['Change']<0,'Volumecolor']=0
  df[['Change','Volumecolor']]

  # 단순 이동평균
  df.loc[:, 'SMA_20'] = ta.sma(df['Close'],length=20)
  df.loc[:, 'SMA_60'] = ta.sma(df['Close'],length=60)

  # 지수 이동평균
  df.loc[:, 'EMA_20'] = ta.ema(df['Close'], 20)
  df.loc[:, 'EMA_60'] = ta.ema(df['Close'], 60)

  df.loc[:, 'RSI_14'] = ta.rsi(df['Close'], length=14)
  df.loc[:, 'RSI_14'] = df['RSI_14'].fillna(0)
  return df

def draw_candlechart(df):
  df['Change']=df['Close']-df['Open']
  df.loc[df['Change']>=0,'Volumecolor']=1
  df.loc[df['Change']<0,'Volumecolor']=0
  df[['Change','Volumecolor']]
  fig = make_subplots(rows=2,
                      cols=1,
                      shared_xaxes=True,
                      )

  fig = make_subplots(rows=2, cols=1)

  fig.add_trace(
      go.Candlestick(x=df.index,
                  open=df['Open'],
                  high=df['High'],
                  low=df['Low'],
                  close=df['Close'],
                  increasing_line_color= 'red', decreasing_line_color= 'blue',
                  name='주가'
                  ),
                  row=1, col=1
  )

  fig.add_trace(
      go.Bar(x=df.index, y=df['Volume'],
            marker=dict(color=df['Volumecolor'],colorscale='BlueRed'),
            name='거래량'),
      row=2, col=1
  )

  fig.update(layout_xaxis_rangeslider_visible=False)

  fig.show()

def draw_sma_ema(name,df):
  df = cal_sma_ema_rsi(df)

  fig = make_subplots(rows=2,
                      cols=1,
                      shared_xaxes=True,
                      vertical_spacing=0.1)

  fig = make_subplots(rows=2, cols=1)

  fig.add_trace(
      go.Candlestick(x=df.index,
                  open=df['Open'],
                  high=df['High'],
                  low=df['Low'],
                  close=df['Close'],
                  increasing_line_color= 'red', decreasing_line_color= 'blue',
                  name='주가'
                  ),
                  row=1, col=1
  )

  fig.add_trace(
      go.Scatter(x=df.index, y=df['SMA_20'],
                line=dict(color="#b66aa0"),
                name='SMA_20'),
      row=1, col=1
  )

  fig.add_trace(
      go.Scatter(x=df.index, y=df['SMA_60'],
                line=dict(color="#414b73"),
                  name='SMA_60'),
      row=1, col=1
  )

  fig.add_trace(
      go.Scatter(x=df.index, y=df['EMA_20'],
                line=dict(color="#8b8b8b"),
                  name='EMA_20'),
      row=1, col=1
  )


  fig.add_trace(
      go.Scatter(x=df.index, y=df['EMA_60'],
                line=dict(color="#e8d887"),
                  name='EMA_60'),
      row=1, col=1
  )

  fig.add_trace(
      go.Bar(x=df.index, y=df['Volume'],
            marker=dict(color=df['Volumecolor'],colorscale='BlueRed'),
            name='거래량'),
      row=2, col=1
  )

  fig.update(layout_xaxis_rangeslider_visible=False)
  fig.update_layout(title_text=name)

  fig.show()


# RSI
def draw_rsi(stock_name, stock_data):
  fig = plt.subplots(figsize=(10, 6), sharex=True)
  gs = gridspec.GridSpec(nrows=2, ncols=1, height_ratios=[2, 1])

  # 주가 나타내기
  ax1 = plt.subplot(gs[0])
  ax1 = stock_data['Close'].plot()
  ax1.set_xlabel('')
  ax1.axes.xaxis.set_ticks([])
  plt.title(stock_name)

  # RSI 나타내기
  ax2 = plt.subplot(gs[1])
  ax2 = stock_data['RSI_14'].plot(color='black', ylim=[0, 100])
  ax2.axhline(y=70, color='r', linestyle='-')
  ax2.axhline(y=30, color='r', linestyle='-')
  ax2.set_xlabel
  plt.subplots_adjust(wspace=0, hspace=0)

  plt.show()
  return stock_data

def draw_bbbands(stock_name, stock_data):
  band = ta.bbands(stock_data["Close"], length=20, std=2)
  bb = pd.concat([band[['BBL_20_2.0', 'BBM_20_2.0', 'BBU_20_2.0']], stock_data['Close']], axis = 1)
  bb.columns = ['Lower Band', 'Mid Band', 'Upper Band', 'Close']
  bb.plot(figsize=(10, 6),
          color={
              'Upper Band': 'red',
              'Lower Band': 'blue',
              'Mid Band': 'green',
              'Close': 'black'
          })
  plt.title(stock_name)
  plt.show()

모든 반도체 종목에 대해 캔들차트 그리기

In [ ]:
name = '자람테크놀로지'
draw_sma_ema(name,get_stockdata(stock[name]))

In [ ]:
# for name, value in stock.items():
#   draw_bbbands(name,draw_rsi(name,draw_ema(name,draw_sma(name,match_dfshape(get_stockdata(value),'2021-01-01','2024-03-13')))))

In [ ]:
draw_sma_ema(name,get_stockdata('123010'))

In [ ]:
for name, value in stock.items():
  print(name)
  draw_sma_ema(name,get_stockdata(value))
# print(get_stockdata('005930'))

In [ ]:
stock_df[datetime.strptime('2023-03','%Y-%m')+relativedelta(months=1):]

In [ ]:
#df.loc['2022/12'].loc['매출액']
stock_df.loc[:idx_date,'매출액'] = df.loc['2022/12','매출액']

# 예측

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable

import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.manual_seed(128)
MinMaxScaler = MinMaxScaler()


In [ ]:
class LSTM(nn.Module):
  def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
    super(LSTM, self).__init__()
    self.num_classes = num_classes #number of classes
    self.num_layers = num_layers #number of layers
    self.input_size = input_size #input size
    self.hidden_size = hidden_size #hidden state
    self.seq_length = seq_length #sequence length

    self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                      num_layers=num_layers, batch_first=True) #lstm
    self.fc_1 =  nn.Linear(hidden_size, 128) #fully connected 1
    self.fc = nn.Linear(128, num_classes) #fully connected last layer

    self.relu = nn.ReLU()

  def forward(self,x):
    h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(device) #hidden state
    c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(device) #internal state
    # Propagate input through LSTM

    output, (hn, cn) = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state

    hn = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next
    out = self.relu(hn)
    out = self.fc_1(out) #first Dense
    out = self.relu(out) #relu
    out = self.fc(out) #Final Output

    return out

In [ ]:
class GRU(nn.Module) :
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length) :
        super(GRU, self).__init__()
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length

        self.gru = nn.GRU(input_size=input_size,hidden_size=hidden_size,
                         num_layers=num_layers,batch_first=True)
        self.fc_1 = nn.Linear(hidden_size, 128)
        self.fc = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x) :
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))
        output, (hn) = self.gru(x, (h_0))
        hn = hn.view(-1, self.hidden_size)
        out = self.relu(hn)
        out = self.fc_1(out)
        out = self.relu(out)
        out = self.fc(out)
        return out

In [ ]:
num_epochs = 30000 #1000 epochs
learning_rate = 0.00001 #0.001 lr

input_size = 5 #number of features
hidden_size = 2 #number of features in hidden state
num_layers = 1 #number of stacked lstm layers

num_classes = 1 #number of output classes

In [ ]:
lstm = LSTM(num_classes, input_size, hidden_size, num_layers, X_train_tensors_final.shape[1]).to(device)

loss_function = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(lstm1.parameters(), lr=learning_rate)  # adam optimizer

In [ ]:
for epoch in range(num_epochs):
  outputs = lstm.forward(X_train_tensors_final.to(device)) #forward pass
  optimizer.zero_grad() #caluclate the gradient, manually setting to 0

  # obtain the loss function
  loss = loss_function(outputs, y_train_tensors.to(device))

  loss.backward() #calculates the loss of the loss function

  optimizer.step() #improve from loss, i.e backprop
  if epoch % 100 == 0:
    print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))

In [ ]:
df_X_ss = ss.transform(df.drop(columns='Volume'))
df_y_mm = mm.transform(df.iloc[:, 5:6])

df_X_ss = Variable(torch.Tensor(df_X_ss)) #converting to Tensors
df_y_mm = Variable(torch.Tensor(df_y_mm))
#reshaping the dataset
df_X_ss = torch.reshape(df_X_ss, (df_X_ss.shape[0], 1, df_X_ss.shape[1]))

In [ ]:
train_predict = lstm(df_X_ss.to(device))#forward pass
data_predict = train_predict.data.detach().cpu().numpy() #numpy conversion
dataY_plot = df_y_mm.data.numpy()

data_predict = mm.inverse_transform(data_predict) #reverse transformation
dataY_plot = mm.inverse_transform(dataY_plot)
plt.figure(figsize=(10,6)) #plotting
plt.axvline(x=4500, c='r', linestyle='--') #size of the training set

plt.plot(dataY_plot, label='Actuall Data') #actual plot
plt.plot(data_predict, label='Predicted Data') #predicted plot
plt.title('Time-Series Prediction')
plt.legend()
plt.show()